## 객체를 파일로 저장하고 불러오기

### `pickle`

In [1]:
data = {}
data[1] = {'no' : 1, 'subject' : '안녕 피클', 'content' : '피클은 매우 간단합니다'}

In [2]:
import pickle

with open('data.p', 'wb') as f:
    pickle.dump(data, f)

In [3]:
with open('data.p','rb') as f:
    data = pickle.load(f)
    
data

{1: {'no': 1, 'subject': '안녕 피클', 'content': '피클은 매우 간단합니다'}}

In [4]:
def get_all_data():
    try :
        with open("data.p", 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        return {}
    
def add_data(no, subject, content):
    data = get_all_data()
    ## assert no not in data
    data[no] = {'no' : no, 'subject' : subject, 'content' : content}
    with open('data.p', 'wb') as f:
        pickle.dump(data, f)
        
def get_data(no):
    data = get_all_data()
    return data[no]

## 데이터 저장
add_data(1, '안녕 피클', '피클은 매우 간단합니다')

## 데이터 조회
data = get_data(1)
print(data['no'])
print(data['subject'])
print(data['content'])

1
안녕 피클
피클은 매우 간단합니다


## 객체 변경에 따른 오류 방지

### `copyreg`

In [5]:
import pickle

class Student:
    def __init__(self, name, age):
        self.name = name
        self.age = age

a = Student('임철희', 27)

with open('student.p', 'wb') as f:
    pickle.dump(a, f)

In [6]:
with open('student.p','rb') as f:
    student = pickle.load(f)
    
print(student.name)     ## 임철희 출력

임철희


In [7]:
class Student:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.dummy = 'dummy'            ## dummy 속성 추가

In [8]:
with open('student.p', 'rb') as f:
    student = pickle.load(f)            ## dummy 속성이 생성되기 전에 저장한 pickle 데이터를 읽으면
    
print(student.dummy)                    ## dummy 속성을 찾을 수 없다는 오류 발생 

AttributeError: 'Student' object has no attribute 'dummy'

In [9]:
import copyreg

class Student:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
def pickle_student(student):
    kwargs = student.__dict__
    return unpickle_student, (kwargs, )

def unpickle_student(kwargs):
    return Student(**kwargs)

copyreg.pickle(Student, pickle_student)

a = Student('임철희', 27)
with open('student.p', 'wb') as f:
    pickle.dump(a, f)

In [10]:
class Student:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.dummy = 'dummy'
        
def pickle_student(student):
    kwargs = student.__dict__
    return unpickle_student, (kwargs, )

def unpickle_student(kwargs):
    return Student(**kwargs)

copyreg.pickle(Student, pickle_student)

with open('student.p', 'rb') as f:
    student = pickle.load(f)            ## unpickle_student() 함수를 호출한다
    
print(student.dummy)

dummy


## 딕셔너리를 파일로 저장하려면

### `shelve`

In [11]:
def save(key, value):
    """ key 에 대응하는 value를 저장한다. """
    pass

def get(key):
    """key 에 저장된 value를 반환한다"""
    pass

In [12]:
import shelve
def save(key, value):
    with shelve.open('shelve.dat') as d:
        d[key] = value
    
def get(key):
    with shelve.open('shelve.dat') as d:
        return d[key]
    
save('number', [1,2,3,4,5])
print(get('number'))

[1, 2, 3, 4, 5]


## 블로그 데이터 저장하기

### `sqlite3`

- 데이터 베이스 접속하기

`connect()`

In [13]:
import sqlite3
conn = sqlite3.connect('blog.db')

- 테이블 생성하기

`CREATE TABLE 테이블명 (...)`<br>
`execute()` 함수를 호출하여 실행

In [14]:
c = conn.cursor()
c.execute('''CREATE TABLE blog (id integer PRIMARY KEY, subject text, content text, date text)''')

- 데이터 입력하기

`INSERT INTO 테이블명 VALUES (...)`

In [15]:
c.execute("INSERT INTO blog VALUES(1, '첫번째블로그', ' 첫 번째 블로그입니다.', '20230802')")

In [16]:
c.execute("INSERT INTO blog VALUES(2, '두번째블로그', ' 두 번째 블로그입니다.', '20230802')")

In [17]:
_id = 3
subject = "세번째블로그"
content = "세 번째 블로그입니다"
date = "20230802"
c.execute("INSERT INTO blog VALUES (%d, '%s', '%s', '%s')" % (_id, subject, content, date))

In [18]:
_id = 4
subject = "네번째블로그"
content = "네 번째 블로그입니다"
date = "20230802"
c.execute("INSERT INTO blog VALUES (?, ?, ?, ?)", (_id, subject, content, date))

In [19]:
c.execute("INSERT INTO blog VALUES (:id, :subject, :content, :date)", {"id": 5, "subject" : "다섯번째블로그", "content": "다섯 번째 블로그입니다", "date": "20230802"})

- 데이터 조회하기

`SELECT ... FROM 테이블명 ...`

In [20]:
c.execute('SELECT * FROM blog')
all = c.fetchall()
print(all)

[(1, '첫번째블로그', ' 첫 번째 블로그입니다.', '20230802'), (2, '두번째블로그', ' 두 번째 블로그입니다.', '20230802'), (3, '세번째블로그', '세 번째 블로그입니다', '20230802'), (4, '네번째블로그', '네 번째 블로그입니다', '20230802'), (5, '다섯번째블로그', '다섯 번째 블로그입니다', '20230802')]


In [21]:
c.execute('SELECT * FROM blog')
one = c.fetchone()
print(one)

(1, '첫번째블로그', ' 첫 번째 블로그입니다.', '20230802')


- 데이터 수정과 삭제

`UPDATE 테이블명 SET ...` : 수정 <br>
`DELETE FROM 테이블명 ...`: 삭제

In [22]:
c.execute("UPDATE blog SET subject='최초의블로그' WHERE id=1")

In [23]:
c.execute('SELECT * FROM blog WHERE id=1')
one = c.fetchone()
print(one)

(1, '최초의블로그', ' 첫 번째 블로그입니다.', '20230802')


In [24]:
c.execute('DELETE FROM blog WHERE id=5')        ## id 안쓰면 전체 삭제임

- 데이터 저장과 취소

`conn.commit()` : 저장<br>
`conn.rollback()` : 취소

In [25]:
conn.commit()

In [26]:
conn.rollback()

- 데이터베이스 접속 종료

`conn.close()` : 종료

In [27]:
conn.close()

- 블로그 데이터 모델

In [28]:
import sqlite3
## 조회
def get_blog_list():
    conn = sqlite3.connect('blog.db')
    c = conn.cursor()
    c.execute("SELECT * FROM blog")
    result = c.fetchall()
    conn.close()
    return result

In [31]:
get_blog_list()

[(1, '최초의블로그', ' 첫 번째 블로그입니다.', '20230802'),
 (2, '두번째블로그', ' 두 번째 블로그입니다.', '20230802'),
 (3, '세번째블로그', '세 번째 블로그입니다', '20230802'),
 (4, '네번째블로그', '네 번째 블로그입니다', '20230802'),
 (5, '자동 증가', 'id값이 자동 증가되어 입력됩니다.', '20230802')]

In [35]:
## 딕셔너리로 전체 조회
def get_blog_list():
    conn = sqlite3.connect('blog.db')
    conn.row_factory = sqlite3.Row
    c = conn.cursor()
    c.execute("SELECT * FROM blog")
    result = c.fetchall()
    conn.close()
    return result

In [33]:
for blog in get_blog_list():
    print(blog)
    print(blog["subject"])

최초의블로그
두번째블로그
세번째블로그
네번째블로그
자동 증가


In [34]:
import time
## 블로그 생성
def add_blog(subject, content):
    conn = sqlite3.connect('blog.db')
    conn.row_factory = sqlite3.Row
    c = conn.cursor()
    today = time.strftime('%Y%m%d')
    c.execute("INSERT INTO blog (subject, content, date) VALUES (?,?,?)",
              (subject, content, today))
    conn.commit()
    conn.close()
    

In [38]:
## 블로그 읽기
def read_blog(_id):
    conn = sqlite3.connect('blog.db')
    conn.row_factory = sqlite3.Row
    c = conn.cursor()
    c.execute("SELECT * FROM blog WHERE id=?", (_id,))
    result = c.fetchone()
    conn.close()
    return result

In [42]:
## 블로그 수정
def modify_blog(_id, subject, content):
    conn = sqlite3.connect('blog.db')
    conn.row_factory = sqlite3.Row
    c = conn.cursor()
    c.execute("UPDATE blog SET subject=?, content=?, WHERE id=?",
              (subject, content, _id))
    conn.commit()
    conn.close()

In [43]:
## 블로그 삭제
def remove_blog(_id):
    conn = sqlite3.connect('blog.db')
    conn.row_factory = sqlite3.Row
    c = conn.cursor()
    c.execute("DELETE FROM blog WHERE id=?", (_id,))
    conn.commit()
    conn.close()

In [44]:
## connect 나 close 같이 반복하여 사용하는것을 데코레이터 해서 사용하기

def with_cursor(original_func):
    def wrapper(*args, **kwargs):
        conn = sqlite3.connect('blog.db')
        conn.row_factory = sqlite3.Row
        c = conn.cursor()
        rv = original_func(c, *args, **kwargs)
        conn.commit()
        conn.close()
        return rv
    return wrapper

In [45]:
## 최종 합본(.py파일 같이 만듬)

import sqlite3
import time

def with_cursor(original_func):
    def wrapper(*args, **kwargs):
        conn = sqlite3.connect('blog.db')
        conn.row_factory = sqlite3.Row
        c = conn.cursor()
        rv = original_func(c, *args, **kwargs)
        conn.commit()
        conn.close()
        return rv
    return wrapper

@with_cursor
def get_blog_list(c):
    c.execute("SELECT * FROM blog")
    return c.fetchall()

@with_cursor
def add_blog(c, subject, content):
    c.execute("INSERT INTO blog (subject, content, date) VALUES (?,?,?)",
              (subject, content, time.strftime('%Y%m%d')))
    
@with_cursor
def read_blog(c, _id):
    c.execute("SELECT * FROM blog WHERE id=?", (_id,))
    return c.fetchone()

@with_cursor
def modify_blog(c, _id, subject, content):
    c.execute("UPDATE blog SET subject=?, content=?, WHERE id=?",
              (subject, content, _id))
    
@with_cursor
def remove_blog(c, _id):
    c.execute("DELETE FROM blog WHERE id=?", (_id,))